In [6]:
import os

import io

import csv 

from zipfile import ZipFile

import glob 

import random 

from google.cloud import bigquery

from google.oauth2 import service_account

import sqlite3

In [7]:
# Authentication: these keys will be different on your device, must change to run your own code. 
# Path to connect to GBQ

service_path = "/Users/clairedanicich/Desktop/ADA/wedge/wedge2020/"
service_file = "MSBA-cfc85a1b18d5.json"
gbq_proj_id = "msba-291619"
gbq_dataset_id = "wedge2"

privatekey = service_path + service_file

# Pass in credentials
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# Establish connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [8]:
#This is the GBQ query used to pull Owners
query = ("Select Distinct card_no,"
"From `msba-291619.wedge2.transArchive*`"
"Where card_no !=3"
)

# Execute using `client.query`
query_job = client.query(query,location = "US",)

In [9]:
# Create a list with the distinct Owners from GBQ tables
owners = []

for idx, row in enumerate(query_job): 
    #since row in GBQ is a tuple, you need to place []
    card_no = row[0] 
    
    owners.append(card_no)

In [14]:
# Pull random sample of 300 owners
random.seed(20201021)#start random generator at same spot
owner_sample = random.choices(owners,k=300) #random list of owners

In [25]:
# Run list comprehension to utilize the random sample of owners and pull the data
query = """Select * From `msba-291619.wedge2.transArchive*` Where card_no in ("""

query = query + ",".join([str(owner) for owner in owner_sample]) + ")"

query_job = client.query(query,location = "US",)

In [26]:
#header names
headers = ["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype",
           "trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax",
           "taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided",
           "percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff",
           "numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display",
           "receipt","card_no","store","branch","match_id","trans_id"]

In [27]:
# Write outfiles
with open("owner_query.txt", "w") as outfile: 
    # Join the headers to the new outfile 
    outfile.write(",".join(headers) + "\n")
    for row in query_job: 
        outfile.write(",".join([str(item) for item in row])+"\n")